In [1]:
from itertools import repeat
import pickle5 as pickle
import pandas as pd
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
import rioxarray as rio
import rasterio
import numpy as np
import geopandas as gpd
from multiprocessing import Pool
import os
from rasterio.merge import merge
from itertools import product
from glob import glob

In [2]:
from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']
id_grid=conf['config']['id-grid-pilot']


In [3]:
import dill
with open('ml_output/07_final_model/model_final.pickle', 'rb') as handle:
    model_=dill.load(handle)
best_model=model_[0]
best_fs=model_[1]

In [4]:
best_model

StackingClassifier(cv=3,
                   estimators=[('cat_wona_pipe',
                                Pipeline(steps=[('transform_cat_wo',
                                                 FunctionTransformer(func=<function <lambda> at 0x7fcf84644940>)),
                                                ('catboost_wo_na',
                                                 <catboost.core.CatBoostClassifier object at 0x7fcf846a0dc0>)])),
                               ('lgbm_wona_pipe',
                                Pipeline(steps=[('transform_lgbm_wo',
                                                 FunctionTransformer(func=<function <lambda> at 0x7fceb6b3fee0>)),
                                                ('l...
                                                                random_seed=1234))])),
                               ('rf_wa_pipe',
                                Pipeline(steps=[('transform_rf_wa',
                                                 FunctionTransformer(func=<function <lambda> at 0x7fceb6945ee0>)),
                                                ('randomforest_w_a',
                                                 RandomForestClassifier(bootstrap=False,
                                                                        class_weight='balanced',
                                                                        max_depth=8,
                                                                        max_leaf_nodes=12,
                                                                        min_samples_leaf=36,
                                                                        min_samples_split=53,
                                                                        n_jobs=30,
                                                                        random_state=1234))]))],
                   final_estimator=LogisticRegression())

In [9]:
def parallel_classify_raster(idgrid):
    global best_model,best_fs,pic_
    
    name_image='duatahun_'+str(idgrid)+'_QALPN1_PakKus_sentinel2_10m.tif'
    #cutting_raster_temp(idgrid,name_image)
    dt_=glob('ml_output/08_class_result/raster_temp/'+idgrid+'*.tif')
    with Pool(20) as p:
        p.map(pool_paralle,dt_)
    ls_file=glob('ml_output/08_class_result/temp/*.tif')
    src_files_to_mosaic = []
    tif_name='ml_output/08_class_result/result/classified_'+str(idgrid)+'.tif'
    ls_y=[y for y in ls_file if y.replace('classified_','').split('_')[0]==str(idgrid)]
    for ls in ls_y:
        src = rasterio.open('ml_output/08_class_result/temp/'+ls)
        src_files_to_mosaic.append(src)
    mosaic, out_trans = merge(src_files_to_mosaic)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff","height": mosaic.shape[1],"width": mosaic.shape[2],"transform": out_trans})
    with rasterio.open(tif_name, "w", **out_meta) as dest:
        dest.write(mosaic)
        print(tif_name, 'Done')

In [10]:
def cutting_raster_temp(idgrid,name_image):
    data_riox=rio.open_rasterio('gs://bps-gcp-bucket/citra-sentinel2/'+pic_+'/'+name_image,
                                masked=True)
    min_x=min(data_riox.x).values+0
    max_x=max(data_riox.x).values+0
    min_y=min(data_riox.y).values+0
    max_y=max(data_riox.y).values+0
    x_=list(np.linspace(min_x,max_x,10))
    y_=list(np.linspace(min_y,max_y,10))
    rangex=range(0,len(x_)-1)
    rangey=range(0,len(y_)-1)
    list_=[]
    for i,j in product(rangex,rangey):
        minx=x_[i]
        miny=y_[j]
        maxx=x_[i+1]
        maxy=y_[j+1]
        geometries = [{
            'type': 'Polygon',
            'coordinates': [[
                [minx, miny],
                [minx, maxy],
                [maxx, maxy],
                [maxx, miny],
                [minx, miny]
            ]]}]
        clipped = data_riox.rio.clip(geometries,all_touched=True)
        clipped.rio.to_raster("ml_output/08_class_result/raster_temp/"+idgrid+"_"+str(i)+"_"+str(j)+".tif", 
                              compress='LZMA', tiled=True, dtype="float64")

In [11]:
def pool_paralle(ls):
    global best_model, best_fs
    data_riox=rio.open_rasterio(ls)
    temp_=data_riox.to_dataframe(name='value').reset_index().fillna(0)
    matrixData=temp_[['x','y']].drop_duplicates()
    for h in temp_.band.unique():
        t_=temp_.loc[temp_.band==h,['x','y','value']].rename(columns={'value':data_riox.long_name[h-1]})
        matrixData=matrixData.merge(t_,how='left')
    matrixData=gpd.GeoDataFrame(matrixData, crs='epsg:4326',
                                        geometry=gpd.points_from_xy(matrixData.x,matrixData.y)).to_crs('epsg:3857')
    matrixData['prediction']=best_model.predict(matrixData[best_fs])
    matrixData['prediction']=matrixData['prediction'].astype(np.int32)
    geo_grid = make_geocube(
                            vector_data=matrixData,
                            measurements=['prediction'],
                            resolution=(-10, 10),
                            rasterize_function=rasterize_points_griddata)
    geo_grid.prediction.rio.write_nodata(0,inplace=True)
    tif_name='classified_'+ls[-15:]+'.tif'
    geo_grid.rio.to_raster('ml_output/08_class_result/temp/'+tif_name)

In [12]:
parallel_classify_raster('ID-3172')

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/ensemble/_base.py:209: UserWarning: Loky-backed parallel loops cannot be called in a multiprocessing, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/home/jupyter-peta_admin/.local/

  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pr

KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
for i in id_grid:
    parallel_classify_raster(i)